In [9]:
import os
import json
from typing import List, Dict, Any
from dotenv import load_dotenv
from openai import OpenAI, OpenAIError
from getpass import getpass

In [5]:
if load_dotenv():
    api_key = os.getenv("OPENAI_API_KEY")
else:
    api_key = getpass("Enter your OpenAI API key:")

if not api_key:
    raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

client = OpenAI(
    api_key=api_key,
)

In [6]:
CHAT_HISTORY_FILE = "chat_history.json"
UNANSWERED_QUESTIONS_FILE = "unanswered_questions.json"

In [7]:
sample_reviews = [
    "Great course! The guest lectures from industry leaders were very inspiring.",
    "Trop de théorie, pas assez de pratique. Les exemples étaient intéressants mais souvent trop abstraits.",
    "The weekly presentations were challenging but helped solidify the course content.",
    "Assignments were overwhelming at times, especially with group projects happening simultaneously.",
    "Beaucoup trop de lectures chaque semaine. J'aurais préféré plus de travaux pratiques.",
    "The course offers a lot of insights into AI product management, but I feel some content could be more up-to-date.",
    "Excellent professor with clear explanations, but the pace of the course was too fast.",
    "Les séances de coaching hebdomadaires ont été utiles pour clarifier des concepts difficiles.",
    "Too much focus on theory rather than practical applications. Felt more like a lecture in AI basics than product management.",
    "Overall, a solid course with knowledgeable instructors and relevant content. However, group coordination was difficult.",
    "I felt lost during the agile development module; more examples would have been helpful.",
    "Bon cours, mais difficile de tout suivre avec autant de travail. Les lectures étaient longues.",
    "The emphasis on real-world cases is helpful, but the workload was too intense for a semester course.",
    "Les professeurs étaient engagés et passionnés, ce qui a rendu les sessions plus intéressantes.",
    "The product launch and marketing modules were excellent. I learned a lot about releasing AI products.",
    "This course was harder than expected. The weekly engagement was intense, and I wish I had more time for self-study.",
    "Le module sur la découverte des produits AI était particulièrement bien fait et très intéressant.",
    "Very challenging course but highly informative. The lectures were well-structured and covered AI trends.",
    "La charge de travail était beaucoup trop lourde, surtout les projets de groupe et les présentations.",
    "Great mix of theory and practical insights. Weekly presentations forced us to stay on track.",
    "The guest speakers were top-notch, sharing insights from real industry applications of AI product management.",
    "Complex material but valuable insights into AI product strategy and development.",
    "A must-take course for anyone looking to go into AI product management. A bit intense but very rewarding.",
    "Les discussions de groupe ont aidé à approfondir la matière, mais la charge de travail était énorme.",
    "A lot of focus on the business side of AI rather than the technical side, which was refreshing.",
    "I struggled with the heavy reading load. Would have liked more guidance on which readings were essential.",
    "Trop de théorie, j'aurais aimé plus d'exemples pratiques, en particulier pour le module de développement agile.",
    "In-depth and informative, but the course could benefit from more interactive elements.",
    "The professor's approach to product strategy was insightful. The projects were a bit overwhelming, though.",
    "Course content was dense, covering product management and AI applications. Real-life case studies were very useful.",
    "The course was demanding but I came away with a good understanding of AI product management.",
    "The emphasis on practical exercises was great, but the simulation games were confusing at times.",
    "I appreciated the structured approach, but the grading criteria could have been clearer.",
    "Intéressant mais la charge de travail était bien trop lourde. Peu de temps pour approfondir les lectures.",
    "Enjoyed the group projects, but coordinating with teammates took a lot of time.",
    "I learned a lot about AI product vision and market fit. However, more guidance during projects would have been helpful.",
    "The course introduced me to a lot of new concepts, but I felt rushed at times.",
    "Professeur très compétent, mais les explications étaient parfois difficiles à suivre pour des débutants.",
    "Loved the case studies and the hands-on approach, though it was intense. Final project was challenging but rewarding.",
    "The agile methodologies module was really insightful, but there was too much overlap with other courses I am taking.",
    "Focus on communication and teamwork in the class was great preparation for real-world AI product management.",
    "Un bon équilibre entre la théorie et la pratique, mais les exigences pour les présentations hebdomadaires étaient élevées.",
    "The assignments pushed us to apply concepts, but I wish there were more guidance on how to structure them.",
    "Les études de cas étaient excellentes et les invités de l'industrie ont donné de bonnes perspectives.",
    "Very comprehensive course, though the final presentation requirements were unclear.",
    "One of the best courses I’ve taken, but I wish the material were updated to include more recent AI innovations.",
    "Les lectures hebdomadaires étaient parfois répétitives, j'aurais aimé un peu plus de variété dans les sujets.",
    "Weekly class discussions were useful, but some assignments felt redundant.",
    "Challenging content, especially for students without a strong AI background. Guest lectures helped clarify complex topics.",
    "The focus on human and ethical dimensions of AI was enlightening but could be developed further.",
    "Some of the real-world examples were outdated. Otherwise, a solid course for aspiring product managers.",
    "I wish the class had covered more about launching AI products in smaller companies.",
    "The instructor’s experience in the industry was evident, which made the lectures very engaging and relevant.",
    "Good balance of strategy and product management, but the workload could be spaced out better."
]

In [21]:
def load_json(file_path: str) -> Any:
    """
    Loads JSON data from a file. Returns an empty list if file does not exist.
    """
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    return []

def save_json(data: Any, file_path: str) -> None:
    """
    Saves JSON data to a file.
    """
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

def initialize_files():
    """
    Initializes chat history and unanswered questions files if they don't exist.
    """
    if not os.path.exists(CHAT_HISTORY_FILE):
        save_json([], CHAT_HISTORY_FILE)
    if not os.path.exists(UNANSWERED_QUESTIONS_FILE):
        save_json([], UNANSWERED_QUESTIONS_FILE)

def load_chat_history() -> List[Dict[str, str]]:
    """
    Loads the chat history from the JSON file.
    """
    return load_json(CHAT_HISTORY_FILE)

def save_chat_history(history: List[Dict[str, str]]) -> None:
    """
    Saves the chat history to the JSON file.
    """
    save_json(history, CHAT_HISTORY_FILE)

def load_unanswered_questions() -> List[str]:
    """
    Loads unanswered questions from the JSON file.
    """
    return load_json(UNANSWERED_QUESTIONS_FILE)

def save_unanswered_question(question: str) -> None:
    """
    Saves an unanswered question to the JSON file.
    """
    questions = load_unanswered_questions()
    questions.append(question)
    save_json(questions, UNANSWERED_QUESTIONS_FILE)

def construct_messages(history: List[Dict[str, str]], user_message: str, n_pros: int, n_cons: int, include_summary: bool) -> List[Dict[str, Any]]:
    """
    Constructs the messages list for the OpenAI API based on chat history and user message.
    """
    # System message to define assistant's behavior
    system_message = {
        "role": "system",
        "content": (
            "You are an AI assistant specialized in answering questions about the AI Product Management course. "
            "You have access to the following course reviews:\n" +
            "\n".join(sample_reviews) +
            "\n\nBased on the reviews and the ongoing conversation, provide a concise and accurate answer to the user's question. "
            "Avoid hallucinating any information not present in the reviews.\n"
        )
    }

    # Initialize messages with system message
    messages = [system_message]

    # Append conversation history
    for message in history:
        messages.append({
            "role": message["role"],
            "content": message["content"]
        })

    # Append the new user message
    messages.append({
        "role": "user",
        "content": user_message
    })

    return messages

def send_message(client: openai.OpenAI, user_message: str, n_pros: int, n_cons: int, include_summary: bool, reviews: List[str]) -> str:
    """
    Sends a message to the AI assistant and returns the response.
    """
    # Load chat history
    history = load_chat_history()
    
    # Construct messages list
    messages = construct_messages(history, user_message, n_pros, n_cons, include_summary)
    
    try:
        # Call OpenAI API
        response = client.chat.completions.create(
            model="gpt-4",  # Replace with the correct model name, e.g., "gpt-4"
            messages=messages,
            max_tokens=1500,  # Adjust as needed
            temperature=0.3,
            stream=False
        )
        assistant_message = response.choices[0].message.content.strip()
        
        # Update chat history
        history.append({"role": "user", "content": user_message})
        history.append({"role": "assistant", "content": assistant_message})
        save_chat_history(history)
        
        # Check if the assistant generated a question
        # Assume that if the assistant can't answer, it includes "Question for students:" in the response
        if "Question for students:" in assistant_message:
            # Extract the question
            question = assistant_message.split("Question for students:")[-1].strip()
            save_unanswered_question(question)
            # Optionally, modify the assistant's message to exclude the question
            assistant_message = assistant_message.replace(f"Question for students: {question}", "").strip()
        
        return assistant_message
    
    except OpenAIError as e:
        return f"An error occurred while processing the chat: {str(e)}"

def start_chat_demo():
    """
    Demonstrates the chat functionality in a script or Jupyter notebook.
    """
    initialize_files()
    
    print("Welcome to the AI Product Management Course Chatbot!")
    print("Type 'exit' to end the chat.\n")
    
    while True:
        try:
            user_input = input("You: ")
        except (KeyboardInterrupt, EOFError):
            print("\nEnding the chat. Goodbye!")
            break

        if user_input.lower() in ["exit", "quit"]:
            print("Ending the chat. Goodbye!")
            break
        
        response = send_message(
            client=client,
            user_message=user_input,
            n_pros=5,
            n_cons=5,
            include_summary=True,
            reviews=sample_reviews
        )
        
        print(f"Assistant: {response}\n")

In [ ]:
start_chat_demo()

Welcome to the AI Product Management Course Chatbot!
Type 'exit' to end the chat.



You:  What about the workload?


Assistant: The workload for the AI Product Management course seems to be quite heavy according to several reviews. Some students found the assignments, especially the group projects, to be overwhelming. The reading load was also mentioned as being intense, with some students wishing for more guidance on which readings were essential. The weekly presentations and discussions, while useful, added to the workload. Some students also mentioned that the workload was too intense for a semester course. However, despite the heavy workload, many students found the course to be very informative and rewarding.



You:  Do TA serve us pizza evenings and hackatons?


Assistant: The course reviews do not provide any information about Teaching Assistants (TAs) serving pizza evenings or organizing hackathons. Could you please clarify if these events were part of the AI Product Management course?

